In [82]:
import base64
import io
from PIL import Image
import cv2
import numpy as np

from keras.models import Model, load_model

In [83]:
data="..."

In [84]:
splitIndex = data.find(',')

In [85]:
data = data[splitIndex+1:]

In [86]:
model = load_model('models/2020_11_23_17_19_13.h5')

In [92]:
def stringToRGB(base64_string):
    imgdata = base64.b64decode(base64_string)
    dataBytesIO = io.BytesIO(imgdata)
    image = Image.open(dataBytesIO)
    return cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)

In [93]:
img = stringToRGB(data)

In [94]:
img.shape

(143, 266, 3)

In [89]:
def stringToRGB(base64_string):
    imgdata = base64.b64decode(base64_string)
    dataBytesIO = io.BytesIO(imgdata)
    image = Image.open(dataBytesIO)
    return image

In [43]:
testimg_resized = cv2.resize(testImg,(336,188))
testimg_resized.shape

(188, 336, 3)

In [44]:
testimg_resized = testimg_resized.astype(np.float32) / 255

In [45]:
testimg_resized = np.expand_dims(testimg_resized, axis=0)

In [46]:
predict = model.predict(testimg_resized)
predict

array([[3.046662]], dtype=float32)

In [49]:
result = np.exp(predict[0][0])

In [50]:
result

21.044981